# A different formulation for a Laplace-Laplace equation system



## Background

Se considera una única superficie que separa las cargas del exterior

**Interior**
$$ \nabla^2 \phi_1 \left(r\right) = -\frac{1}{\epsilon_1} \sum_k q_k \delta \left( r - r_k \right) $$
**Exterior**
$$ \nabla^2 \phi_2 \left(r\right) = 0 $$

con las condiciones de borde 

$$\phi_1 = \phi_2 , \hspace{20pt}
\epsilon_1 \frac{\partial \phi_1}{\partial \textbf{n}} = 
\epsilon_2 \frac{\partial \phi_2}{\partial \textbf{n}} $$

Luego la energía de disolución se puede calcular con la ecuación de toda la vida 

$$ E = \frac{1}{2} \sum_k q_k \phi \left( r_k \right) $$


### Formulación integral


$$
\phi_1 \left(r\right) + 
\int_\Gamma \phi_1  \left(r'\right)
\frac{\partial G_L}{\partial n} \left(r, r'\right) \textrm{ d}r' -
\int_\Gamma G_L \left(r, r'\right)
\frac{\partial \phi_1}{\partial n} \left(r'\right) \textrm{ d}r' = 
\frac{1}{4 \pi \epsilon}\sum_k \frac{q_k}{|r - r_k|}
$$

$$
\phi_2 \left(r\right) - 
\int_\Gamma \phi_2  \left(r'\right)
\frac{\partial \textrm{G}_L}{\partial n} \left(r, r'\right) \textrm{ d}r' +
\int_\Gamma \textrm{G}_L \left(r, r'\right)
\frac{\partial \phi_2}{\partial n}\left(r'\right) \textrm{ d}r' = 0
$$

Se despeja $\phi_1$ de las condiciones de contorno para dejar todo el sistema en términos de la misma variable

$$\phi_2 = \phi_1$$
$$ \frac{\partial \phi_2}{\partial n} = 
  \frac{\epsilon_1}{\epsilon_2} \frac{\partial \phi_1}{\partial n}$$
  
ambas regiones poseen la misma función de Green

$$ \textrm{G}_L \left(r, r'\right) = \frac{1}{4\pi|r-r'|} $$

Al llevar el punto de evaluación a la superficie, y dejando en términos del potencial $\phi_1$ utilizando las condiciones de contorno, 

$$
\frac{\phi}{2} \left(r\right) + 
\int_\Gamma \phi  \left(r'\right)
\frac{\partial \textrm{G}_L}{\partial n} \left(r, r'\right) \textrm{ d}r' -
\int_\Gamma \textrm{G}_L \left(r, r'\right)
\frac{\partial \phi}{\partial n} \left(r'\right) \textrm{ d}r' = 
\frac{1}{4 \pi \epsilon}\sum_k \frac{q_k}{|r - r_k|}
$$

$$
\frac{\phi}{2} \left(r\right) - 
\int_\Gamma \phi  \left(r'\right)
\frac{\partial \textrm{G}_L}{\partial n} \left(r, r'\right) \textrm{ d}r' + 
\frac{\epsilon_1}{\epsilon_2} \int_\Gamma \textrm{G}_L \left(r, r'\right)
\frac{\partial \phi}{\partial n} \left(r'\right) \textrm{ d}r' = 0
$$



### Reducción de la formulación

Como se puede observar, los operadores en ambas ecuaciones son los mismos, ya que representan el mismo campo, y operan sobre la misma superficie, por lo que es posible manipularlos. Al sumar las dos ecuaciones originales, se elimina el operador $K_L$, quedando de la forma:

$$
\frac{\phi}{2} \left(r\right) +
\left( \frac{\epsilon_1}{\epsilon_2} - 1 \right) 
\int_\Gamma \textrm{G}_L \left(r, r'\right)
\frac{\partial \phi}{\partial n} \left(r'\right) \textrm{ d}r' = 
\frac{1}{4 \pi \epsilon}\sum_k \frac{q_k}{|r - r_k|}
$$

El reducir la cantidad de operadores involucrados en el sistema disminuye el tiempo de ensamblado, pero no necesariamente tiene una correlación directa en el tiempo de convergencia del sistema una vez armado.
Luego se deriva la ecuación con respecto a la normal de la superficie

$$
\frac{\partial \phi}{\partial n} \left(r\right) +
\left( \frac{\epsilon_1}{\epsilon_2} - 1 \right) 
\int_\Gamma \frac{\partial \textrm{G}_L \phi}{\partial n \left(r\right)}  \left(r, r'\right) 
\frac{\partial \phi}{\partial n} \left(r'\right) \textrm{ d}r' = 
\frac{1}{4 \pi \epsilon}\sum_k 
\frac{\partial}{\partial n} \left( \frac{q_k}{|r - r_k|} \right)
$$

cabe destacar que la derivada que se le agrega en la integral es con respecto a $r$ y no $r'$, por lo que se es distinto al operador $K_L$ visto en la primera formulación, y se denora $K'_L$ (Adjoint Double Layer).

La ventaja que posee esta formulación es que permite resolver de manera directa el problema, ya que se elimina el espacio $\phi \left(r\right)$ de la ecuación. Una vez obtenido el valor de $\frac{\partial \phi}{\partial n}$ es posible reemplazar en las ecuaciones originales, en este caso se utiliza la ecuación exterior original, y con esto se obtiene otro sistema similar al anterior.

$$
\frac{\phi}{2} \left(r\right) - 
\int_\Gamma \phi \left(r'\right)
\frac{\partial \textrm{G}_L}{\partial n} \left(r, r'\right) \textrm{ d}r' = -
\frac{\epsilon_1}{\epsilon_2} \int_\Gamma \textrm{G}_L \left(r, r'\right)
\frac{\partial \phi}{\partial n} \left(r'\right) \textrm{ d}r' 
$$


## Example

Se estudia el caso de una esfera con un arreglo de 2 cargas, uno en el centro y otro en la mitad del radio

In [1]:
import numpy as np
import bempp.api
import time

ep_in, ep_ex = 4., 80.
x_q, q = np.array([[ 0., 0., 0. ], [0.5, 0., 0.]]), np.array([1., 0.])

grid = bempp.api.shapes.regular_sphere(5)
neumann_space = bempp.api.function_space(grid, "DP", 0)
print "Number of elements: {0}".format(neumann_space.global_dof_count)


Number of elements: 8192


Se resuelve el primer sistema para obtener $\frac{\partial \phi}{\partial n}$

$$ \left[ I + \left( \frac{\epsilon_1}{\epsilon_2} - 1 \right) K'_L \right] \frac{\partial \phi}{\partial n} = 
\frac{-1}{4 \pi \epsilon_1} \sum_k \frac{q_k}{|r - r_k|^3} (r-r_k)\cdot n$$

In [2]:
print "Assembling First Matrix..."
matrix_time = time.time()

from bempp.api.operators.boundary import sparse, laplace
d_identy = sparse.identity(neumann_space, neumann_space, neumann_space)
adj = laplace.adjoint_double_layer(neumann_space, neumann_space, neumann_space)
A = d_identy + (ep_in/ep_ex - 1.)*adj

def charges_dfun(x, n, domain_index, result):
    cook = 1.47
    const = -1./(4.*np.pi*ep_in)
    result[:] = const*np.sum(q*np.dot( x - x_q, n )/(np.linalg.norm( x - x_q, axis=1 )**3))
    
rhs_1 = bempp.api.GridFunction(neumann_space, fun=charges_dfun)

matrix_time = time.time() - matrix_time
print "Assamble time: {:5.2f}".format(matrix_time)


solver_time = time.time()
total_neumann, info =  bempp.api.linalg.iterative_solvers.gmres(A, rhs_1, tol=1e-5)
# total_neumann, info =  bempp.api.linalg.cg(A, rhs_1, tol=1e-5)
solver_time = time.time() - solver_time
print "The linear system was solved in {:5.2f} seconds".format(solver_time)


Assembling First Matrix...
Assamble time:  1.89
The linear system was solved in 13.52 seconds


Con el resultado utilizamos la ecuación exterior original para obtener $\phi$

$$ \left[ \frac{I}{2} - K_L \right] \phi = 
\left[ -\frac{\epsilon_1}{\epsilon_2} V_L \right] \frac{\partial \phi}{\partial n} $$

In [3]:
dirichl_space = bempp.api.function_space(grid, "DP", 0)

print "Assembling Second Matrix..."
matrix_time = time.time()

identy = sparse.identity(dirichl_space, dirichl_space, dirichl_space)
dlp = laplace.double_layer(dirichl_space, dirichl_space, dirichl_space)
B = 0.5*identy - dlp

slp = laplace.single_layer(neumann_space, dirichl_space, dirichl_space)
rhs_2 = ((-ep_in/ep_ex)*slp)*total_neumann

matrix_time = time.time() - matrix_time
print "Assamble time: {:5.2f}".format(matrix_time)


solver_time = time.time()
total_dirichl, info = bempp.api.linalg.iterative_solvers.gmres(B, rhs_2, tol=1e-5)
# total_dirichl, info =  bempp.api.linalg.cg(B, rhs_2, tol=1e-5)
solver_time = time.time() - solver_time
print "The linear system was solved in {:5.2f} seconds".format(solver_time)


Assembling Second Matrix...
Assamble time: 11.65
The linear system was solved in 14.07 seconds


Luego se puede obtener la energía total como se hace usualmente

In [4]:
slp_ev = bempp.api.operators.potential.laplace.single_layer(neumann_space, x_q.transpose())
dlp_ev = bempp.api.operators.potential.laplace.double_layer(dirichl_space, x_q.transpose())
phi_q = slp_ev*total_neumann - dlp_ev*total_dirichl

total_energy = 2*np.pi*332.064*np.sum(q*phi_q).real
print "Total dissolution Energy: {:8.3f} [kcal/mol]".format(total_energy)


Total dissolution Energy:  -26.748 [kcal/mol]
